In [ ]:
import pandas as pd
import warnings
import os

warnings.filterwarnings("ignore")   

base_pagos_efect_path = 'bases/efectividades/BasePagos_Efectividades_20240918_Contrato.xlsx'

dashboard_efect_path = 'dashboard/DASHBOARD_EFECTIVIDADES_SEPTIEMBRE.xlsx'
dashboard_saldos_path = 'dashboard/DASHBOARD_SALDOS_SEPTIEMBRE.xlsx'

dashboard_efect_path = os.path.abspath(dashboard_efect_path)
dashboard_saldos_path = os.path.abspath(dashboard_saldos_path)

print(base_pagos_efect_path)
print(dashboard_efect_path)
print(dashboard_saldos_path)

df_pagos = pd.read_excel(base_pagos_efect_path, sheet_name="BD")

In [ ]:
df_pagos_efect = df_pagos.copy()
df_pagos_saldos = df_pagos.copy()

print(df_pagos_efect.shape)
df_pagos_efect.head(3)

In [ ]:
cols =  ["fecha", "periodo", "dia", "Dia_Util", "codcent", "contrato", "clave", "CapitalSoles", "PagoTotalSoles", "PagoEfectTotalSoles", "PagoEfectTotalSolesAgencia", "PagoEfectTotalSolesAgenciaCont", "Agencia", "desc_tramo", "foco"]
df_pagos_efect = df_pagos_efect[cols]
print(df_pagos_efect.shape)

df_pagos_efect = df_pagos_efect[df_pagos_efect["PagoEfectTotalSolesAgenciaCont"]!=0]
print(df_pagos_efect.shape)

def marca_kst(row):
    if row["clave"] == "KSTBC":
        if row["desc_tramo"] in ["< 01 año", "De 90 a 180 dias", "< 02 años"]:
            return "KSTBC"
        else:
            return "KSTBC_2"
    return ""

df_pagos_efect["MARCA KST"] = df_pagos_efect.apply(marca_kst, axis=1)
df_pagos_efect["codcent"] = df_pagos_efect["codcent"].astype(str)
df_pagos_efect["contrato"] = df_pagos_efect["contrato"].astype(str).str.zfill(20)
df_pagos_efect["cont_18"] = df_pagos_efect["contrato"].apply(lambda x: x[:8] + x[-10:])

cols =  ["fecha", "periodo", "dia", "Dia_Util", "codcent", "contrato", "cont_18", "clave", "CapitalSoles", "PagoTotalSoles", "PagoEfectTotalSoles", "PagoEfectTotalSolesAgencia", "PagoEfectTotalSolesAgenciaCont", "Agencia", "foco", "MARCA KST"]
df_pagos_efect = df_pagos_efect[cols]
df_pagos_efect.to_excel(dashboard_efect_path, index=False)

df_pagos_efect.head(3)

In [4]:
###################################################################################################################

In [ ]:
cols =  ['Agencia', 'clave', 'SEGMENTO_RIESGO', 'AMBITO_RCD_FINAL', 'CapitalSoles']
df_pagos_saldos = df_pagos_saldos[cols]
print(df_pagos_saldos.shape)

df_pagos_saldos.rename(columns={'clave': 'CLAVE', 'Agencia': 'AGENCIA', 'CapitalSoles': 'CAPITAL', 'AMBITO_RCD_FINAL': 'PRODUCTO', 'SEGMENTO_RIESGO': 'SEGMENTO'}, inplace=True)
print(df_pagos_saldos.shape)

df_pagos_saldos = df_pagos_saldos[df_pagos_saldos['AGENCIA'].isin(["ASESCOM RJ", "CLASA MORA", "MORNESE MORA", "SIN AGENCIA"])]

df_pagos_saldos['SEGMENTO'] = df_pagos_saldos['SEGMENTO'].apply(lambda x: 'PYME / EMP MIN' if x != 'PARTICULARES' else x)
df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].str.replace(r'\d+', '', regex=True).str.replace('.', '').str.upper()
df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].str.strip().replace(' ', '')
df_pagos_saldos['CAPITAL'] = df_pagos_saldos['CAPITAL'].round(2)

df_pagos_saldos = df_pagos_saldos.groupby(['AGENCIA', 'CLAVE', 'SEGMENTO', 'PRODUCTO']).agg({'CAPITAL': 'sum'}).reset_index()
df_pagos_saldos.to_excel(dashboard_saldos_path, index=False)

df_pagos_saldos.head(3)

In [6]:
###################################################################################################################

In [7]:
os.startfile(dashboard_efect_path)
os.startfile(dashboard_saldos_path)